# **Spin Dynamics of the cubic Heisenberg antiferromagnet RbMnF3**
#### Author: Martin Mourigal (mourigal@gatech.edu)
#### Date: July 2022
#### Version: Julia 1.7.1

In this tutorial, we will walk through a first example in Su(n)ny and calculate the thermomagnetic and spin dynamical properties of RbMnFe<sub>3</sub>. This material is considered the closest realization of an cubic, spin-isotropic Heisenberg antiferromagnet and was studied in detail by many famous neutron scattering and magnetism experts, in particular for its critical properties in vicinity to T<sub>N</sub> = 83 K.


[1] C G Windsor and R W H Stevenson 1966 Proc. Phys. Soc. 87 501 (https://doi.org/10.1088/0370-1328/87/2/319)   
[2] D A Tennant et al 1994 J. Phys.: Condens. Matter 6 10341 (https://doi.org/10.1088/0953-8984/6/47/017)

## **Setting up Julia with the necessary packages**

Important ressources to get started:   
https://github.com/SunnySuite/Sunny.jl/blob/main/GettingStarted.md          
https://sunnysuite.github.io/Sunny.jl/dev/library/#Library
https://github.com/ddahlbom/SunnyTutorials/blob/main/tutorials/FeI2_tutorial.ipynb

In [ ]:
## === Packages ===

using Sunny
using Plots, ProgressMeter, Statistics, Formatting, StaticArrays, LinearAlgebra, DelimitedFiles, OffsetArrays
using GLMakie

## **Setting up the Crystal Structure**

### **Method 1: Only specify the magnetic atoms**

We will start by defining a crystal structure by hand, using the lattice parameters of RbMnFe<sub>3</sub> reported as a = 4.218 Å IN [2] and ignoring the non-magnetic atoms in the unit-cell. This is achieved using the `Crystal` function. Note that Su(n)ny automatically recognizes the space-group of the structure and the point group for inequivalent atoms in the basis. As a result, Method 1 is **dangerous** for compounds for which non-magnetic atoms lower the symmetry compared to the magnetic sub-lattice(s). This can be aleviated by specifying an additional space group parameter when defining `Crystal`. Note that for RbMnFe<sub>3</sub>, however, Rb and Mn atoms are on symmetry positions of the 221 space group and thus Method 1 yields correct results.

In [32]:
## === Define a simple cubic lattice occupied by the Mn atoms in RbMnF3 ===

# Unit cell: lattice vectors [Here simple cubic] 
lat_vecs = lattice_vectors(4.218, 4.218, 4.218, 90.0, 90.0, 90.0);  
# Unit cell: fractional coordinates for atoms in basis [Here Mn at (0,0,0)]
bas_vecs = [[0.0000000, 0.000000, 0.0000000]]; 
# Unit cell: type of atoms in the unit cell basis [Here only one atom in basis]
bas_typs = ["Mn"]; 
# Creates the crystal
xtal_cub = Crystal(lat_vecs, bas_vecs; types=bas_typs);

## ===  Access and information and visualization  ===

# This returns crystalographic information for that Crystal
display(xtal_cub); 
# This plots the crystal in a nice 3D interface and allows to draw bonds between atoms up to the distance specified in Å
view_crystal(xtal_cub, 5.0) 

# One can ensure a desired space group, here artificially lowering the symmetry to P-1
lat_spg = 1;
xtal_cub_lowsym = Crystal(lat_vecs, bas_vecs, lat_spg; types=bas_typs);
display(xtal_cub_lowsym); 

Crystal
HM symbol 'P m -3 m' (221)
Lattice params a=4.218, b=4.218, c=4.218, α=90°, β=90°, γ=90°
Cell volume 75.04
Type 'Mn', Wyckoff 1a (point group 'm-3m'):
   1. [0, 0, 0]


Crystal
HM symbol 'P 1' (1)
Lattice params a=4.218, b=4.218, c=4.218, α=90°, β=90°, γ=90°
Cell volume 75.04
Type 'Mn':
   1. [0, 0, 0]


### **Method 2a: Specific the entire chemical unit-cell by hand**

Because Su(n)ny properly performs symmetry analysis for bilinear exchange interactions, it is important to provide all atoms in the unit-cell, this can be accomplish directly, or by using a .cif file. Given the picture here:
![RbMnF3_Structure](images/RbMnF3_Structure.png)


In [33]:
## === Define the crystal structure of RbMnF3 by looking at the above figure ===

lat_vecs = lattice_vectors(4.218, 4.218, 4.218, 90.0, 90.0, 90.0);  
bas_vecs = [[0.0000000, 0.000000, 0.0000000],
            [0.5000000, 0.500000, 0.5000000],
            [0.5000000, 0.000000, 0.0000000],
            [0.0000000, 0.500000, 0.0000000],
            [0.0000000, 0.000000, 0.5000000]]; 
bas_typs = ["Mn","Rb","F","F","F"]; 
xtal_rbmnf3a = Crystal(lat_vecs, bas_vecs; types=bas_typs);

# Access and information and visualization 
display(xtal_rbmnf3a); 
# By selecting a distance of 0.0, only atoms in the given unit-cell are plotted:
view_crystal(xtal_rbmnf3a, 0.0) 


Crystal
HM symbol 'P m -3 m' (221)
Lattice params a=4.218, b=4.218, c=4.218, α=90°, β=90°, γ=90°
Cell volume 75.04
Type 'Mn', Wyckoff 1a (point group 'm-3m'):
   1. [0, 0, 0]
Type 'Rb', Wyckoff 1b (point group 'm-3m'):
   2. [0.5, 0.5, 0.5]
Type 'F', Wyckoff 3d (point group '4/mm.m'):
   3. [0.5, 0, 0]
   4. [0, 0.5, 0]
   5. [0, 0, 0.5]


Sunny.CrystalViewer("<!DOCTYPE html>\n<html>\n\n<head>\n  <title>Sunny Visuals</title>\n\n  <meta name=\"viewport\" content=\"width=device-width, initial-scale=1\">\n  <meta charset=\"UTF-8\" />\n</head>\n\n<body style=\"\n    margin: 0;\n    overflow: hidden;\n    text-align: center;\n    font-size: 12px;\n    font-family: Sans-Serif;\n    color: #444;\n\">\n\n<!-- Might want to copy HTML styles from Jupyter notebook cells here:\n    https://jupyterbook.org/en/stable/interactive/interactive.html\n-->\n<style type=\"text/css\">\n    #flex-container-c0d9c740e8b1{\n        display: flex;\n        margin: 10px;\n        min-height: 400px;\n        min-width: 400px;\n        /* max-width: 800px; */\n    }\n    #widget-container-c0d9c740e8b1 {\n        flex: 0.5;\n        background-color: #EEEEEE;\n        max-width: 200px;\n        border: thin solid black;\n    }\n    #widget-container-c0d9c740e8b1:first-child {\n        margin-right: 10px;\n    }\n    #scene-container-c0d9c740e8b1{\n        flex: 1.0;\n        margin-left:0;\n        background:#000000;\n        border: thin solid black;\n        align-self: center;\n    }\n    /* For an alternative approach to this, see https://stackoverflow.com/a/45046955/500314 */\n    #scene-container-c0d9c740e8b1 canvas{\n        height:100% !important;\n        width:100% !important;\n    }\n</style>\n\n<div id=\"flex-container-c0d9c740e8b1\">\n    <div id=\"widget-container-c0d9c740e8b1\">\n        <!-- Widgets inserted here at runtime. -->\n    </div>\n    <div id=\"scene-container-c0d9c740e8b1\">\n        <!-- <canvas> inserted here at runtime. -->\n    </div>\n</div>\n\n<script id=\"data-c0d9c740e8b1\" type=\"application/json\">{\n\"cellTypes\":    [\"Mn\",\"Rb\",\"F\",\"F\",\"F\"],\n\"bondColors\":   [],\n\"bondLabels\":   [],\n\"bondTypeIds\":  [],\n\"bondVecs\":     [],\n\"lattVecs\":     [[4.218,0.0,0.0],[0.0,4.218,0.0],[0.0,0.0,4.218]],\n\"basisVecs\":    [[0.0,0.0,0.0],[2.1089999999999995,2.1089999999999995,2.1089999999999995],[2.1089999999999995,0.0,0.0],[0.0,2.1089999999999995,0.0],[0.0,0.0,2.1089999999999995]],\n\"lattCells\":    [1,1,1],\n\"atomsPerCell\": 5\n}</script>\n\n<!-- Insert either a link to the Javascript file (development mode) or the\nactual Javascript content (production mode). -->\n<script >'use strict';\n\n(async function () {\n    const THREE = await import(\"https://cdn.skypack.dev/three@0.132.2\");\n    const OrbitControls = (await import(\"https://cdn.skypack.dev/three@0.132.2/examples/jsm/controls/OrbitControls.js\")).OrbitControls;\n\n    // In development, a default key is useful.\n    let key = \"0\";\n    // In production, this line will be replaced to define `key` as a unique\n    // string identifier. This uniqueness is necessary to support multiple\n    // visualizations within the same web page.\n    key = 'c0d9c740e8b1';\n\n    // Get a reference to the container element that will hold our scene\n    const container = document.querySelector(`#scene-container-\${key}`);\n\n    function showError(err) {\n        container.innerHTML = `<div class=\"error\" style=\"color:red; font-size:20px;\">\${err}</div>`;\n        throw err;\n    }\n\n    try {\n        // create a Scene\n        const scene = new THREE.Scene();\n        scene.background = new THREE.Color(\"#EEEEEE\");\n\n        // read in data from Sunny \n        const data = JSON.parse(document.getElementById(`data-\${key}`).textContent);\n        const cellTypes = data.cellTypes;\n        const bondColors = data.bondColors;\n        const bondLabels = data.bondLabels;\n        const bondTypeIds = data.bondTypeIds;\n        const bondVecs = data.bondVecs;\n        const lattVecs = data.lattVecs;\n        const basisVecs = data.basisVecs;\n        const lattCells = data.lattCells;\n        const atomsPerCell = data.atomsPerCell;\n\n        // CPKM colors for all atoms\n        var atomColors = {\n            \"H\":\"#FFFFFF\",  \"He\":\"#D9FFFF\", \"Li\":\"#CC80FF\", \"Be\":\"#C2FF00\",\n    

### **Method 2b: Use of .cif file**

Obtain a .cif file, for instance from: https://icsd.fiz-karlsruhe.de. In some instances, it is necessary to specify the parameter `symprec` depending on the number of significant digitis in the atoms coordinate in the .cif file. Note that the .cif used from ICSD has made a different choice of the origin of the unti-cell. We will keep the two crystals to show they yield equivalent results.

In [34]:
## === Define the crystal structure of RbMnF3 by importing a .cif file ===

xtal_rbmnf3b = Crystal("RbMnF3_ICSD#43722.cif",symprec=0.001);
display(xtal_rbmnf3b); # This gives exactly the same result as above with Mn in 1b position

Crystal
HM symbol 'P m -3 m' (221)
Lattice params a=4.25, b=4.25, c=4.25, α=90°, β=90°, γ=90°
Cell volume 76.77
Type 'Rb1+', Wyckoff 1a (point group 'm-3m'):
   1. [0, 0, 0]
Type 'Mn2+', Wyckoff 1b (point group 'm-3m'):
   2. [0.5, 0.5, 0.5]
Type 'F1-', Wyckoff 3c (point group '4/mm.m'):
   3. [0.5, 0.5, 0]
   4. [0.5, 0, 0.5]
   5. [0, 0.5, 0.5]


### **Setting up sub-crystals**

For methods 2a and 2b, the Rb and F atoms in the structure are non-magnetic and we need to hide them before decorating the structure with exchange interactions. This is achieved with the `subcrystal` function.

In [35]:
## === Defining Subcrystals for the Crystals defined in Method 2a and 2b ==

# Note that we have defined the two crystals in slightly different ways
xtal_rbmnf3a = subcrystal(xtal_rbmnf3a,"Mn");
display(xtal_rbmnf3a);
xtal_rbmnf3b = subcrystal(xtal_rbmnf3b,"Mn2+");
display(xtal_rbmnf3b);

Crystal
HM symbol 'P m -3 m' (221)
Lattice params a=4.218, b=4.218, c=4.218, α=90°, β=90°, γ=90°
Cell volume 75.04
Type 'Mn', Wyckoff 1a (point group 'm-3m'):
   1. [0, 0, 0]


WARNING, atoms are being renumbered.


Crystal
HM symbol 'P m -3 m' (221)
Lattice params a=4.25, b=4.25, c=4.25, α=90°, β=90°, γ=90°
Cell volume 76.77
Type 'Mn2+', Wyckoff 1b (point group 'm-3m'):
   1. [0.5, 0.5, 0.5]


### **Setting up the Spin System**
We will now create a `SpinSystem` associated with this `Crystal`. To do so, we need to look at possible equivalent exchange interactions. 

In [36]:
print_bond_table(xtal_rbmnf3a, 6.0);
print_bond_table(xtal_rbmnf3b, 6.0);
print_bond_table(xtal_cub_lowsym, 6.0);
print_allowed_anisotropy(xtal_rbmnf3b, 1);

Atom 1, type 'Mn', position [0, 0, 0], multiplicity 1
Allowed single-ion anisotropy or g-tensor: | A  0  0 |
                                           | 0  A  0 |
                                           | 0  0  A |

Bond(1, 1, [1, 0, 0])
Distance 4.218, coordination 6
Connects 'Mn' at [0, 0, 0] to 'Mn' at [1, 0, 0]
Allowed exchange matrix: | A  0  0 |
                         | 0  B  0 |
                         | 0  0  B |

Bond(1, 1, [1, 1, 0])
Distance 5.9652, coordination 12
Connects 'Mn' at [0, 0, 0] to 'Mn' at [1, 1, 0]
Allowed exchange matrix: | A  C  0 |
                         | C  A  0 |
                         | 0  0  B |

Atom 1, type 'Mn2+', position [0.5, 0.5, 0.5], multiplicity 1
Allowed single-ion anisotropy or g-tensor: | A  0  0 |
                                           | 0  A  0 |
                                           | 0  0  A |

Bond(1, 1, [1, 0, 0])
Distance 4.25, coordination 6
Connects 'Mn2+' at [0.5, 0.5, 0.5] to 'Mn2+' at [1.5, 0.5, 0.5]
Allowed 